## 6COM1034 Concurrency - Practical
# 02 - Semaphores and Mutexes

Python's `threading` module has a nice collection of tools for concurrent programming. Last week we learned about the essential component - the `Thread` class (and, the `Process` class in the `multiprocessing` module). Today we are going to learn about some of the tools it offers for process synchronisation.

## Learning outcomes

 1. Using semaphores in concurrent Python: the `Semaphore` class
 2. Binary semaphores: the `Lock` class
 3. Mutexes: the `RLock` class
 

## 1. Semaphores: the Producer/Consumer pattern 

We'll play with the Producer/Consumer Pattern that you have seen in the lecture.

The producer makes items and puts them into an array to store them. The storage has limited space, and we want only to be 5 items in storage at maximum. The producer can only make new items if there is space in storage. 

The consumer takes items from the storage array. The consumer can only take items if the storage array is not empty.

We'll use a `list` for storage and `Semaphore` objects from the `threading` class to synchronise the consumer and producer threads. 

In [14]:
import threading as th # python threading 
import random 

storage = [] # this is our storage list

storage_size = 5

We will use two __`Semaphore`__ objects to represent the state of the storage:

1. `space_left`: Indicates whether there's space left in storage. Initialised with the storage size. 
2. `items_left`: Indicates whether there are items left in storage. Initialised with `0`.

In [15]:
space_left = th.Semaphore(storage_size)
items_left = th.Semaphore(0)

Finally, we also need to make sure that only one thread accesses the queue at one time. We use a __binary semaphore__ for this. In Python, this is implemented by the `Lock` class.

In [10]:
use_storage = th.Lock()

We will need functions `produce` and `consume` that the producer and consumer threads will run. 

The `produce` function should do the following:

2. Make sure that there's space in storage, by acquiring the `space_left` semaphore.
1. Acquire the `Lock` on the storage, i.e. acquire the `use_storage` binary semaphore.
3. Put an item into storage.
5. Release the `use_storage` lock. 
5. Indicate that there is one more item in storage, by releasing the `items_left` semaphore.
4. `sleep()` for a little while (producing is exhausting!! ;) ).

5. Repeat from step 1.

In addition, we'll stop after producing 10 items.

In [16]:
import time
import random

def produce(space_left, items_left, use_storage):
    global storage
    prod_count = 0
    while(prod_count < 10):
        space_left.acquire()
        use_storage.acquire()
        item = prod_count
        storage.append(item)
        prod_count += 1
        print("Producer made item '{}'. {} items left in storage.".format(item, len(storage)))
        use_storage.release()    
        items_left.release()
        time.sleep(random.random()*0.25) # sleep for at most half a second



Likewise, the `consume` function should:

2. Acquire the `items_left` semaphore to make sure that there is an item in storage.
1. Acquire the lock on the storage.
3. Get an item from storage.
5. Release the `space_left` semaphore, since we just made space for another item in storage.
6. Release the lock on the storage.
7. Repeat from step 1. 
2. `sleep()` for a little while. 


We'll stop after having consumed 10 items.

In [17]:
def consume(space_left, items_left, use_storage):
    global storage
    cons_count = 0
    while(cons_count < 10):
        items_left.acquire()
        use_storage.acquire()
        item = storage.pop(0)
        cons_count += 1
        print("Consumer ate  item '{}'. {} items left in storage.".format(item, len(storage)))
        use_storage.release()    
        space_left.release()
        time.sleep(random.random()*0.51) # sleep for at most half a second



<hr/>
### Exercise 1

Time to put it all together and run the example. Please run the code below to create the threads and run them. 

Note how the sequence of producing and consuming items interleaves.

In [18]:
producer = th.Thread(target=produce, args=(space_left, items_left, use_storage))
consumer = th.Thread(target=consume, args=(space_left, items_left, use_storage))
producer.start()
consumer.start()
producer.join()
consumer.join()

Producer made item '0'. 1 items left in storage.
Consumer ate  item '0'. 0 items left in storage.
Producer made item '1'. 1 items left in storage.
Consumer ate  item '1'. 0 items left in storage.
Producer made item '2'. 1 items left in storage.
Producer made item '3'. 2 items left in storage.
Consumer ate  item '2'. 1 items left in storage.
Producer made item '4'. 2 items left in storage.
Producer made item '5'. 3 items left in storage.
Producer made item '6'. 4 items left in storage.
Producer made item '7'. 5 items left in storage.
Consumer ate  item '3'. 4 items left in storage.
Consumer ate  item '4'. 3 items left in storage.
Producer made item '8'. 4 items left in storage.
Consumer ate  item '5'. 3 items left in storage.
Producer made item '9'. 4 items left in storage.
Consumer ate  item '6'. 3 items left in storage.
Consumer ate  item '7'. 2 items left in storage.
Consumer ate  item '8'. 1 items left in storage.
Consumer ate  item '9'. 0 items left in storage.



<hr/>

### Exercise 2

Modify the code of the first cell in this notebook so that the storage has space only for one item. Execute all cells from the top to make all code pick up this change. Run the code from exercise 1 several times and observe the sequence of items that are produced and consumed. Do you see any interleaving? If not, why not?

__NOTE__: It can happen that your program stalls due to a programming error. You will notice that the indicator next to the cell that executes the threaded code says __`In [*]:`__ and stays like this indefinitely. In this case, __press the `0` key twice. __ This will reset your kernel. You need to execute the notebook again from the beginning. 

<br/>
<hr/>

## 2. Mutex: The rubber chicken

Next we will implement the "rubber chicken" metaphor that we discussed in the tutorial. From the lecture slides:

<blockquote>To prevent interruptions during work meetings, suppose the meeting has a rule that a person holding a token is the only one allowed to talk.  A rubber chicken is kept in the meeting for just such occasions. The person holding the chicken is the only person who is allowed to talk. If you don't hold the chicken you cannot speak. You can only indicate that you want the chicken by holding out your hand and wait until you get it before you speak. Once you have finished speaking, you can hand the chicken back to the moderator who will hand it to the next person to speak. This ensures that people do not speak over each other, and also have their own space to talk.</blockquote>

For an implementation we need:

2. Several persons, implemented as concurrent threads, that try to contribute something to the meeting.
1. A mutex that stands for the rubber chicken.

Let's get started! First we need names of people attending the meeting and their contribution. (Note: Any similarity with actual business meetings is purely coincidental.)

In [19]:
names_and_phrases = {"Linda": "We need to do more with less.",
                     "Jeff": "I see your point.",
                     "Jack": "I'm cautiously optimistic.",
                     "Pat": "It is what it is.",
                     "George": "Take it to the next level.",
                     "Janis": "It's a low-hanging fruit.",
                     "Helen": "We need to think outside the box!"}

Next we implement the `say_something` function, that the threads will use. Out of politeness each speaker waits for a fraction of a second before they try to grab the rubber chicken.

In [20]:
def say_something(rubber_chicken, name, phrase):
    time.sleep(random.random()*0.5) # politely wait a little before trying to grab the rubber chicken
    rubber_chicken.acquire()
    print("{} says: '{}'".format(name, phrase))
    rubber_chicken.release()

Python's `threading` module provides the mutex functionality in the form of the `RLock` class. Let's use this:

In [22]:
rubber_chicken = th.RLock()

Finally, we make all threads, one for each person, `start()` them, and `join()` them when they're finished.

In [25]:
threads = []
for name,phrase in names_and_phrases.items():
    threads.append(th.Thread(target=say_something, args=(rubber_chicken, name, phrase)))

for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

Janis says: 'It's a low-hanging fruit.'
Helen says: 'We need to think outside the box!'
Pat says: 'It is what it is.'
George says: 'Take it to the next level.'
Jack says: 'I'm cautiously optimistic.'
Linda says: 'We need to do more with less.'
Jeff says: 'I see your point.'


<hr/>
### Exercise 3

Run the above example a couple of times. 

You should observe that the sequence is changing every time. This is due to the small wait period at the beginning of the `say_something` function. 

<br/>
<hr/>

## 2.1 Mutexes are exclusive

The key functionality of a Mutex is that only the thread that acquired it can release it.

Please take a minute to look up the documentation of the `RLock`:

(Reminder: The __'`q`'__ key closes the documentation window again).

In [25]:
th.RLock?

The key sentence here is 
<blockquote>A reentrant lock __must be released by the thread that acquired it__.</blockquote>

This is what sets it apart from the `Lock` class, which is essentially a binary semaphore. 

But let's see what happens when we violate that principle. Let's consider a minor change in our little business scenario:

<blockquote>In our small business, the investors have recently appointed a new CEO, Bruce, who should get the business to make more money fast. Bruce is sometimes a bit rude and doesn't want to wait until people release the rubber chicken. He tries to grab the chicken and say his phrase before the others have finished. </blockquote>

You might have guessed it already - Bruce is going to have a hard time trying this with a mutex. But let's see how it goes. First we need a `say_something()` function for Bruce that tries to do the `threading`-equivalent of grabbing the chicken, namely to release the mutex before acquiring it. 

In [26]:
def bruce_says_something(rubber_chicken, name, phrase):
    time.sleep(random.random()*0.5) 
    rubber_chicken.release() # this is the act of prying the chicken from someone else's hands
    rubber_chicken.acquire()
    print("{} says: '{}'".format(name, phrase))
    rubber_chicken.release()

Let's make threads again as above, making a special one for Bruce, and run them:

In [28]:
threads = []
for name,phrase in names_and_phrases.items():
    threads.append(th.Thread(target=say_something, args=(rubber_chicken, name, phrase)))
# here comes Bruce
threads.append(th.Thread(name="Bruce", target=bruce_says_something, args=(rubber_chicken, "Bruce", "You're fired!")))
    
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

Pat says: 'It is what it is.'
Helen says: 'We need to think outside the box!'
George says: 'Take it to the next level.'
Janis says: 'It's a low-hanging fruit.'
Jack says: 'I'm cautiously optimistic.'
Linda says: 'We need to do more with less.'
Jeff says: 'I see your point.'


Exception in thread Bruce:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-26-d5a5e2f7a4a7>", line 3, in bruce_says_something
    rubber_chicken.release() # this is the act of prying the chicken from someone else's hands
RuntimeError: cannot release un-acquired lock



### Exercise 4


Observe what happens to Bruce. Have a look at the error message. This is Python's way to politely remind Bruce to wait for his turn to speak!